In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
from keras.applications.vgg19 import VGG19
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten

import os
from tqdm import tqdm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import cv2
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory|

In [75]:
df_train = pd.read_csv(r"C:\Users\SATYAM\Downloads\Diabetic-Retinopathy-with-CNN-master\DR Data\labels\trainLabels15.csv")

In [76]:
df_train.head()

,image,level
0,10_left,0
1,10_right,0
2,13_left,0
3,13_right,0
4,15_left,1


In [77]:
targets_series = pd.Series(df_train['level'])
one_hot = pd.get_dummies(targets_series, sparse = True)

In [78]:
one_hot_labels = np.asarray(one_hot)

In [79]:
im_size1 = 224
im_size2 = 224

In [80]:
x_train = []
y_train = []
x_test = []

In [89]:
#  i = 0 
#  for f, breed in tqdm(df_train.values):
#     if type(cv2.imread("C:\\Users\\SATYAM\\Downloads\\Diabetic-Retinopathy-with-CNN-master\\DR Data\\resized train 15".format(f)))==type(None):
#         continue
#     else:
#         img = cv2.imread("C:\\Users\\SATYAM\\Downloads\\Diabetic-Retinopathy-with-CNN-master\\DR Data\\resized train 15.jpeg".format(f))
#         label = one_hot_labels[i]
#         x_train.append(cv2.resize(img, (im_size1, im_size2)))
#         y_train.append(label)
#         i += 1
# np.save('x_train2',x_train)
# np.save('y_train2',y_train)
# print('Done')


x_train = []
y_train = []
i = 0

for f, breed in tqdm(df_train.values):
    if type(cv2.imread("C:\\Users\\SATYAM\\Downloads\\Diabetic-Retinopathy-with-CNN-master\\DR Data\\resized train 15\\{}.jpg".format(f))) == type(None):
        continue
    else:
        img = cv2.imread("C:\\Users\\SATYAM\\Downloads\\Diabetic-Retinopathy-with-CNN-master\\DR Data\\resized train 15\\{}.jpg".format(f))
        label = one_hot_labels[i]  # Assuming one_hot_labels is correctly defined
        x_train.append(cv2.resize(img, (im_size1, im_size2)))
        y_train.append(label)
        i += 1

np.save('x_train2.npy', x_train)
np.save('y_train2.npy', y_train)
print('Done')


100%|██████████| 35126/35126 [05:06<00:00, 114.60it/s]  


Done


In [92]:
#type(cv2.imread("C:\\Users\\SATYAM\\Downloads\\Diabetic-Retinopathy-with-CNN-master\\DR Data\\resized train 15\\{}.jpg".format(f)))


NoneType

In [93]:
x_train = np.load('x_train2.npy')
y_train = np.load('y_train2.npy')

In [94]:
y_train_raw = np.array(y_train, np.uint8)
x_train_raw = np.array(x_train, np.float32) / 255.

In [95]:
print(x_train_raw.shape)
print(y_train_raw.shape)

(15934, 224, 224, 3)
(15934, 5)


In [96]:
X_train, X_valid, Y_train, Y_valid = train_test_split(x_train_raw, y_train_raw, test_size=0.2, train_size=0.8, random_state=1)


In [97]:
num_class = y_train_raw.shape[1]

In [98]:
from keras.applications.resnet50 import ResNet50

In [99]:
base_model = ResNet50(weights = None, include_top=False, input_shape=(im_size1, im_size2, 3))

# Add a new top layer
x = base_model.output
x = Flatten()(x)
x = Dropout(0.2)(x)
x = Dense(32, activation='relu')(x)
x = Dense(16, activation='relu')(x)
predictions = Dense(num_class, activation='softmax')(x)

# This is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# First: train only the top layers (which were randomly initialized)
#for layer in base_model.layers:
#    layer.trainable = False

model.compile(loss='categorical_crossentropy', 
              optimizer='rmsprop', 
              metrics=['accuracy'])

callbacks_list = [keras.callbacks.EarlyStopping(monitor='val_acc', verbose=1)]
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 26,799,621 (102.23 MB)

 Trainable params: 26,746,501 (102.03 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [100]:
model.fit(X_train, Y_train, epochs=5, validation_data=(X_valid, Y_valid), verbose=1)

Epoch 1/5
399/399 ━━━━━━━━━━━━━━━━━━━━ 1562s 4s/step - accuracy: 0.6360 - loss: 4.1702 - val_accuracy: 0.7170 - val_loss: 0.9290
Epoch 2/5
399/399 ━━━━━━━━━━━━━━━━━━━━ 1237s 3s/step - accuracy: 0.7305 - loss: 0.9119 - val_accuracy: 0.7170 - val_loss: 0.9266
Epoch 3/5
399/399 ━━━━━━━━━━━━━━━━━━━━ 1230s 3s/step - accuracy: 0.7340 - loss: 0.8861 - val_accuracy: 0.7170 - val_loss: 0.9125
Epoch 4/5
399/399 ━━━━━━━━━━━━━━━━━━━━ 1254s 3s/step - accuracy: 0.7320 - loss: 0.8886 - val_accuracy: 0.7170 - val_loss: 0.9119
Epoch 5/5
399/399 ━━━━━━━━━━━━━━━━━━━━ 1616s 4s/step - accuracy: 0.7308 - loss: 0.8933 - val_accuracy: 0.7170 - val_loss: 0.9106


In [101]:
model.compile(loss='categorical_crossentropy', 
              optimizer='sgd', 
              metrics=['accuracy'])

In [102]:
model.fit(X_train, Y_train, epochs=2, validation_data=(X_valid, Y_valid), verbose=1)

Epoch 1/2
399/399 ━━━━━━━━━━━━━━━━━━━━ 1314s 3s/step - accuracy: 0.7369 - loss: 0.8762 - val_accuracy: 0.7170 - val_loss: 0.9104
Epoch 2/2
399/399 ━━━━━━━━━━━━━━━━━━━━ 1220s 3s/step - accuracy: 0.7326 - loss: 0.8783 - val_accuracy: 0.7170 - val_loss: 0.9140
